<a href="https://colab.research.google.com/github/GabzBarbosa/Analise-NPS/blob/main/Classifica%C3%A7%C3%A3o_(moda_casual_esporte).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pillow numpy scikit-learn tqdm



In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import requests
from io import BytesIO


In [ ]:
df = pd.read_csv("/content/Extração Analise imagem MODA (1).csv")
# Remove espaços extras nos nomes das colunas
df.columns = df.columns.str.strip()
df.head()



,sku,nome,marca,url
0,D7A-651H-219-34,Sandalia Beira Rio Feminina Salto Bloco Confor...,Beira Rio,https://netshoes-image-store.s3.amazonaws.com/...
1,NR3-64CU-219-01,Bolsa Feminina Vizzano Baguete Grande Camurça ...,Vizzano,https://netshoes-image-store.s3.amazonaws.com/...
2,D83-54JJ-205-41,Tênis Denim Joe Ultra Light Democrta 526101 Of...,Democrata,https://netshoes-image-store.s3.amazonaws.com/...
3,C4F-A001-012-01,Bolsa Unissex RDN Travel GymBag Sport Pequena ...,RDN Travel,https://netshoes-image-store.s3.amazonaws.com/...
4,AVL-31A8-006-01,Estojo Box Rebecca Bonbon 26234,Clio,https://netshoes-image-store.s3.amazonaws.com/...


In [ ]:
def carregar_imagem(caminho):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}

        if caminho.startswith("http"):
            response = requests.get(caminho, headers=headers, timeout=10)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            img = Image.open(caminho).convert("RGB")

        img = img.resize((128, 128))
        return np.array(img).flatten()

    except Exception:
        return None


In [ ]:
# ===== TERMOS DE CLASSIFICAÇÃO =====

moda_terms = [
    "salto", "sandália", "bota", "scarpin", "festa", "social", "verniz",
    "moda", "fashion", "tendência", "estiloso", "estilosa",
    "elegante", "clássico", "sofisticado", "moderno", "premium",
    "statement", "exclusivo", "edição limitada",
    "sapatilha", "loafer", "alpargata",
    "salto alto", "salto fino", "salto agulha", "salto bloco",
    "plataforma", "mule", "peep toe", "slingback",
    "glitter", "metalizado", "strass", "salto", "sandália", "bota", "scarpin", "festa", "social", "verniz", 'moda', 'fashion', 'tendência', 'estiloso', 'estilosa', 'elegante', 'clássico', 'sofisticado', 'moderno', 'premium', 'statement', 'exclusivo', 'edição limitada', 'calçados_femininos', 'salto', 'salto alto', 'salto fino', 'salto agulha', 'salto bloco', 'plataforma', 'meia pata', 'scarpin', 'sandália de tira', 'sandália strappy', 'sandália lace-up', 'sandália gladiadora', 'mule', 'peep toe', 'open toe', 'slingback', 'anabela', 'salto dourado', 'salto brilhante', 'pedraria', 'glitter', 'verniz', 'cetim', 'metalizado',"sapatilha", "loafer", "alpargata", 'strass'
]

casual_terms = [
    "tênis", "casual", "camiseta", "dia a dia", "conforto"
]

esporte_terms = [
    "running", "corrida", "treino", "training", "gym", "workout",
    "performance", "sport", "esportivo", "amortecimento",
    "crossfit", "academia", "fitness", "trail", "outdoor", "corrida", "run", "racer", "racing", "treino de corrida", "performance", "speed", "pace", "maratona", "meia maratona", "longão", "velocidade", "trilha", "trail", "trail running", "amortecimento", "espuma responsiva", "respirável", "respiráveis", "leve", "lightweight", "estabilidade", "suporte", "pisada neutra", "pisada pronada", "pisada supinada", "tênis esportivo", "running shoes", "training shoes", "cross training", "crossfit", "stability shoes", "amortecimento", "cushioning", "sola em borracha", "sola antiderrapante", "sola tratorada", "entressola", "EVA", "mesh", "knit", "treino", "training", "workout", "gym", "musculação", "funcional", "HIIT", "cross training", "crossfit", "resistência", "força", "condicionamento", "halter", "dumbbell", "kettlebell", "barra", "anilhas", "cordas", "elástico de resistência", "faixa elástica", "luva de treino", "munhequeira", "basquete", "basketball", "NBA", "quadra", "high-top", "cano alto", "sola de tração", "grip", "shoot", "drible", "rebote", "cesta", "uniforme de basquete", "bola de basquete", "aro", "backboard", "futebol", "soccer", "futsal", "society", "chuteira", "trava", "trava alta", "trava baixa", "grama sintética", "campo", "quadra", "futebol de salão", "caneleira", "meião", "goleiro", "defesa", "ataque", "chute", "drible", "bola de futebol", "ciclismo", "bike", "bicicleta", "MTB", "mountain bike", "speed", "road", "pedal", "sapatilha de ciclismo", "catraca", "capacete", "jersey", "bermuda com forro", "bib", "luva de ciclismo", "vôlei", "volei", "vôlei de praia", "volei de praia", "beach tennis", "raquete", "bola de vôlei", "areia", "rede", "amortecimento lateral", "esportivo", "sport", "performance", "outdoor", "adventure", "resistência", "alto impacto", "baixo impacto", "velocidade", "suporte", "estabilidade", "aderência", "conforto esportivo", "respirável", "ventilado", "peça técnica", "linha esportiva", "camiseta esportiva", "short esportivo", "bermuda running", "legging esportiva", "top esportivo", "jaqueta corta-vento", "windbreaker", "térmica", "dry fit", "Dri-FIT", "techfit", "tecido respirável", "compressão", "segunda pele", "antiodor", "UV protection", "yoga", "pilates", "tapete de yoga", "mat", "alongamento", "equilíbrio", "flexibilidade", "mindfulness", "low impact"
]


In [ ]:
def classificar_nome(nome, marca):
    texto = f"{nome} {marca}".lower()

    if any(t in texto for t in moda_terms):
        return "moda"
    if any(t in texto for t in esporte_terms):
        return "esporte"
    if any(t in texto for t in casual_terms):
        return "casual"

    return "indefinido"


In [ ]:
def classificar_imagem(caminho):
    vetor = carregar_imagem(caminho)
    if vetor is None:
        return "imagem inválida"

    media_pixel = vetor.mean()

    if media_pixel < 80:
        return "moda"
    elif media_pixel > 180:
        return "casual"
    else:
        return "esporte"


In [ ]:
def validar_imagem(vetor):
    if vetor is None:
        return "imagem_invalida"

    if vetor.mean() < 30:
        return "imagem_muito_escura"

    if vetor.mean() > 240:
        return "imagem_muito_clara"

    return "imagem_ok"


In [ ]:
categorias_final = []
validacoes = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processando produtos"):
    # Classificação por texto
    nome_cat = classificar_nome(row["nome"], row["marca"])
    img_cat = classificar_imagem(row["url"])

    # Fallback para imagem
    if nome_cat == "indefinido":
        categorias_final.append(img_cat)
    else:
        categorias_final.append(nome_cat)

    # Validação de imagem
    vetor = carregar_imagem(row["url"])
    validacoes.append(validar_imagem(vetor))


Processando produtos: 100%|██████████| 6448/6448 [3:05:44<00:00,  1.73s/it]


In [ ]:
df["categoria"] = categorias_final
df["validacao_imagem"] = validacoes

df.to_csv("resultado_final.csv", index=False)
df.head()


,sku,nome,marca,url,categoria,validacao_imagem
0,D7A-651H-219-34,Sandalia Beira Rio Feminina Salto Bloco Confor...,Beira Rio,https://netshoes-image-store.s3.amazonaws.com/...,moda,imagem_ok
1,NR3-64CU-219-01,Bolsa Feminina Vizzano Baguete Grande Camurça ...,Vizzano,https://netshoes-image-store.s3.amazonaws.com/...,casual,imagem_ok
2,D83-54JJ-205-41,Tênis Denim Joe Ultra Light Democrta 526101 Of...,Democrata,https://netshoes-image-store.s3.amazonaws.com/...,casual,imagem_muito_clara
3,C4F-A001-012-01,Bolsa Unissex RDN Travel GymBag Sport Pequena ...,RDN Travel,https://netshoes-image-store.s3.amazonaws.com/...,esporte,imagem_ok
4,AVL-31A8-006-01,Estojo Box Rebecca Bonbon 26234,Clio,https://netshoes-image-store.s3.amazonaws.com/...,esporte,imagem_ok


In [ ]:
print(len(df))
print(len(categorias_final))
print(len(validacoes))


6448
6448
6448
